# !!! do NOT run all !!!

# Shopify Capstone: Going from raw data in bz2 format to clean data

In [1]:
# import all necessary modules and submodules
import bz2
import datetime
import sys
import numpy
import json
import re
from datetime import timezone
import pandas

In [2]:
# define filenames ranges
periods = ['2015-11','2015-12','2016-01','2016-02','2016-03','2016-04',
           '2016-05','2016-06','2016-07','2016-08','2016-09','2016-10']
namesRS = []
namesRC = []
for period in periods:
    RS_bz2 = 'data/raw/RS_'+period+'.bz2'
    RS_json = 'data/RS_'+period+'.json'
    RC_bz2 = 'data/raw/RC_'+period+'.bz2'
    RC_json = 'data/RC_'+period+'.json'
    namesRS.append([RS_json,RS_bz2])
    namesRC.append([RC_json,RC_bz2])
fileNames = namesRC + namesRS
filesAll = numpy.array(fileNames)
filesRC = filesAll[0:12,0]
filesRS = filesAll[12:24,0]
filesJSON = filesAll[:,0]

In [8]:
for (file_out,file_in) in filesAll:
    start = datetime.datetime.now()
    print(file_in)

data/raw/RC_2015-11.bz2
data/raw/RC_2015-12.bz2
data/raw/RC_2016-01.bz2
data/raw/RC_2016-02.bz2
data/raw/RC_2016-03.bz2
data/raw/RC_2016-04.bz2
data/raw/RC_2016-05.bz2
data/raw/RC_2016-06.bz2
data/raw/RC_2016-07.bz2
data/raw/RC_2016-08.bz2
data/raw/RC_2016-09.bz2
data/raw/RC_2016-10.bz2
data/raw/RS_2015-11.bz2
data/raw/RS_2015-12.bz2
data/raw/RS_2016-01.bz2
data/raw/RS_2016-02.bz2
data/raw/RS_2016-03.bz2
data/raw/RS_2016-04.bz2
data/raw/RS_2016-05.bz2
data/raw/RS_2016-06.bz2
data/raw/RS_2016-07.bz2
data/raw/RS_2016-08.bz2
data/raw/RS_2016-09.bz2
data/raw/RS_2016-10.bz2


### commented out to prevent replacing existing files
# search all files for occurrences of the word "shopify"
for (file_out,file_in) in filesAll:
    start = datetime.datetime.now()
    print(file_in)
    with open(file_out,'w', errors='surrogateescape') as output:
        #progress = 0
        with bz2.open(file_in, 'rt', errors='surrogateescape') as input_:
            for line in input_:
                #progress = progress + 1
                #if progress > 100:
                #    break
                #if progress % 500000 == 0:
                #    print(progress)
                if 'shopify' in line.lower():
                    output.write(line)
        print(datetime.datetime.now()-start)

In [19]:
# removing links from RC files
for file in filesRC:
    print(file)
    with open(file.replace(".json", "-no_links.json"),'w',errors='surrogateescape') as output:
        with open(file,'rt', errors='surrogateescape') as input_:
            for line in input_:
                line_json = json.loads(line)
                body = line_json["body"]
                line_json["body"] = re.sub('\(?(https?://|www.)[^ ,\n"]*',"---LINK---", body, flags=re.I)
                new_line = json.dumps(line_json)+"\n"
                if 'shopify' in new_line.lower():
                    output.write(new_line)

data/RC_2015-11.json
data/RC_2015-12.json
data/RC_2016-01.json
data/RC_2016-02.json
data/RC_2016-03.json
data/RC_2016-04.json
data/RC_2016-05.json
data/RC_2016-06.json
data/RC_2016-07.json
data/RC_2016-08.json
data/RC_2016-09.json
data/RC_2016-10.json


In [20]:
# keep only links with "shopify" in author, author_flair_text, title, selftext, or subreddit in RS files
exceptions = []
for file in filesRS:
    with open(file.replace(".json", "-four_fields.json"),'w',errors='surrogateescape') as output:
        with open(file,'rt', errors='surrogateescape') as input_:
            print(file)
            lines = input_.readlines()
            for line in lines:
                json_line = json.loads(line)
                xstr = lambda obj: "None" if obj is None else str(obj) + "  "
                fields = ["author", "selftext", "title", "subreddit", "author_flair_text"]
                subline = ""
                for field in fields:
                    try:
                        subline = subline + field + ": " + xstr(json_line[field])
                    except Exception as e:
                        exceptions.append(str(e))
                if "shopify" in subline.lower():
                    output.write(line)
print("exceptions:", set(exceptions))

data/RS_2015-11.json
data/RS_2015-12.json
data/RS_2016-01.json
data/RS_2016-02.json
data/RS_2016-03.json
data/RS_2016-04.json
data/RS_2016-05.json
data/RS_2016-06.json
data/RS_2016-07.json
data/RS_2016-08.json
data/RS_2016-09.json
data/RS_2016-10.json
exceptions: {"'subreddit'"}


In [3]:
# defining a list of names for cleanER RS files
filesFourFields = []
for file in filesRS:
    filesFourFields.append(file.replace(".json", "-four_fields.json"))
filesFourFields

['data/RS_2015-11-four_fields.json',
 'data/RS_2015-12-four_fields.json',
 'data/RS_2016-01-four_fields.json',
 'data/RS_2016-02-four_fields.json',
 'data/RS_2016-03-four_fields.json',
 'data/RS_2016-04-four_fields.json',
 'data/RS_2016-05-four_fields.json',
 'data/RS_2016-06-four_fields.json',
 'data/RS_2016-07-four_fields.json',
 'data/RS_2016-08-four_fields.json',
 'data/RS_2016-09-four_fields.json',
 'data/RS_2016-10-four_fields.json']

In [22]:
# removing links from RS files
for file in filesFourFields:
    with open(file.replace("-four_fields.json", "-no_links.json"),'w',errors='surrogateescape') as output:
        with open(file,'rt', errors='surrogateescape') as input_:
            print(file)
            for line in input_:
                line_json = json.loads(line)
                
                title = line_json["title"]
                line_json["title"] = re.sub('\(?(https?://|www.)[^ ,\n"]*',"---LINK---", title, flags=re.I)
                
                selftext = line_json["selftext"]
                line_json["selftext"] = re.sub('\(?(https?://|www.)[^ ,\n"]*',"---LINK---", selftext, flags=re.I)
                
                author_flair_text = str(line_json["author_flair_text"])
                line_json["author_flair_text"] = re.sub('\(?(https?://|www.)[^ ,\n"]*',"---LINK---", author_flair_text, flags=re.I)
                
                try:
                    subreddit = line_json["subreddit"]
                except Exception as e:
                    subreddit = ""
                
                subline = line_json["author_flair_text"] + line_json["selftext"] + line_json["title"] + line_json["author"] + subreddit
                if "shopify" in subline.lower():
                    new_line = json.dumps(line_json)+"\n"
                    output.write(new_line)

data/RS_2015-11-four_fields.json
data/RS_2015-12-four_fields.json
data/RS_2016-01-four_fields.json
data/RS_2016-02-four_fields.json
data/RS_2016-03-four_fields.json
data/RS_2016-04-four_fields.json
data/RS_2016-05-four_fields.json
data/RS_2016-06-four_fields.json
data/RS_2016-07-four_fields.json
data/RS_2016-08-four_fields.json
data/RS_2016-09-four_fields.json
data/RS_2016-10-four_fields.json


In [4]:
# defining a list of names for clean files
filesNoLink = []
for file in filesJSON:
    filesNoLink.append(file.replace(".json", "-no_links.json"))
filesNoLink

['data/RC_2015-11-no_links.json',
 'data/RC_2015-12-no_links.json',
 'data/RC_2016-01-no_links.json',
 'data/RC_2016-02-no_links.json',
 'data/RC_2016-03-no_links.json',
 'data/RC_2016-04-no_links.json',
 'data/RC_2016-05-no_links.json',
 'data/RC_2016-06-no_links.json',
 'data/RC_2016-07-no_links.json',
 'data/RC_2016-08-no_links.json',
 'data/RC_2016-09-no_links.json',
 'data/RC_2016-10-no_links.json',
 'data/RS_2015-11-no_links.json',
 'data/RS_2015-12-no_links.json',
 'data/RS_2016-01-no_links.json',
 'data/RS_2016-02-no_links.json',
 'data/RS_2016-03-no_links.json',
 'data/RS_2016-04-no_links.json',
 'data/RS_2016-05-no_links.json',
 'data/RS_2016-06-no_links.json',
 'data/RS_2016-07-no_links.json',
 'data/RS_2016-08-no_links.json',
 'data/RS_2016-09-no_links.json',
 'data/RS_2016-10-no_links.json']

In [5]:
# find the set of links
links = []
for file in filesNoLink[:12]:
    print(file)
    with open(file,'rt', errors='surrogateescape') as input_:
        for line in input_:
            links.append(json.loads(line)['link_id'])
for file in filesNoLink[12:]:
    print(file)
    with open(file,'rt', errors='surrogateescape') as input_:
        for line in input_:
            links.append(json.loads(line)['name'])
links = sorted(set(links))
print("unique links:", len(links))

data/RC_2015-11-no_links.json
data/RC_2015-12-no_links.json
data/RC_2016-01-no_links.json
data/RC_2016-02-no_links.json
data/RC_2016-03-no_links.json
data/RC_2016-04-no_links.json
data/RC_2016-05-no_links.json
data/RC_2016-06-no_links.json
data/RC_2016-07-no_links.json
data/RC_2016-08-no_links.json
data/RC_2016-09-no_links.json
data/RC_2016-10-no_links.json
data/RS_2015-11-no_links.json
data/RS_2015-12-no_links.json
data/RS_2016-01-no_links.json
data/RS_2016-02-no_links.json
data/RS_2016-03-no_links.json
data/RS_2016-04-no_links.json
data/RS_2016-05-no_links.json
data/RS_2016-06-no_links.json
data/RS_2016-07-no_links.json
data/RS_2016-08-no_links.json
data/RS_2016-09-no_links.json
data/RS_2016-10-no_links.json
unique links: 10350


In [6]:
# find first links of each month (Reddit)
first_links = []
for file in filesAll[12:24,1]:
    print(file)
    progress = 0
    with bz2.open(file,'rt') as input_:
        for line in input_:
            progress = progress + 1
            if progress > 1:
                break
            line_json = json.loads(line)
            name = line_json["name"]
            first_links.append(name)
first_links

data/raw/RS_2015-11.bz2
data/raw/RS_2015-12.bz2
data/raw/RS_2016-01.bz2
data/raw/RS_2016-02.bz2
data/raw/RS_2016-03.bz2
data/raw/RS_2016-04.bz2
data/raw/RS_2016-05.bz2
data/raw/RS_2016-06.bz2
data/raw/RS_2016-07.bz2
data/raw/RS_2016-08.bz2
data/raw/RS_2016-09.bz2
data/raw/RS_2016-10.bz2


['t3_3r0zf4',
 't3_3uxbvg',
 't3_3yyun4',
 't3_43lprb',
 't3_48ddvx',
 't3_4cstvr',
 't3_4h6v0n',
 't3_4lyd4m',
 't3_4qotdt',
 't3_4vjrss',
 't3_50kc60',
 't3_55atoy']

In [7]:
integrator = lambda string: int(string.replace("t3_",""),36)

In [8]:
# define borders as the first link of each month
borders = []
for link in first_links:
    borders.append(integrator(link))
borders

[226794064,
 233342044,
 240131488,
 247916711,
 255926157,
 263365191,
 270738311,
 278739958,
 286692545,
 294855292,
 303279768,
 311234002]

In [9]:
# find indices of first links in each month (our database)
links_base10 = []
for link in links:
    links_base10.append(integrator(link))
last_for_month = []
j = 0
for i in range(0,len(links_base10)-1): 
    if links_base10[i] > borders[j+1]:
        last_for_month.append(i)
        if j < len(borders)-1-1:             #another -1 so that we only run through
            j = j + 1
        else:
            break
last_for_month.append(len(links_base10))    
last_for_month

[817, 1379, 2074, 2861, 3682, 4479, 5313, 6181, 7150, 8190, 9241, 10350]

In [56]:
integrator(links[last_for_month[0]+1])

233366143

The list above means that:



links with indices

0-71 - were posted prior to month 1

72-817 - were posted in month 1

818-1379 - -||- month 2

...

9242-10350 - -||- month 12

In [10]:
# compile the list of search strings by month
search_strings = []
for index in last_for_month:
    search_strings.append(re.compile("|".join(links[:index])))
search_strings[0]

re.compile(r't3_374967|t3_38ye7o|t3_3bgbra|t3_3bq1ii|t3_3bqwm1|t3_3d2syk|t3_3d68w0|t3_3fq1ny|t3_3gi1j6|t3_3goep1|t3_3gzu63|t3_3h49zn|t3_3h5lko|t3_3hgy3u|t3_3iamiw|t3_3id72c|t3_3idgjj|t3_3isfro|t3_3iw2wi|t3_3jbpg8|t3_3jh533|t3_3kth8c|t3_3l6o8j|t3_3lgqyv|t3_3m3l5b|t3_3muv5c|t3_3mxg5d|t3_3n8r3z|t3_3n91a3|t3_3nmdcw|t3_3npd5e|t3_3nq36h|t3_3nvmtp|t3_3o4vn7|t3_3ofarn|t3_3p2cft|t3_3p2ll8|t3_3p9b0v|t3_3pezp1|t3_3pixah|t3_3pj9d5|t3_3pni20|t3_3pnthc|t3_3ppv1z|t3_3pu3l5|t3_3pxdc0|t3_3pxp5f|t3_3q6uwu|t3_3q6w1y|t3_3q75t4|t3_3q8kbu|t3_3qa9ld|t3_3qbtqz|t3_3qc3be|t3_3qcc87|t3_3qcp2y|t3_3qfbkl|t3_3qgplm|t3_3qj7tn|t3_3qkl49|t3_3qme29|t3_3qobx0|t3_3qpeau|t3_3qq4n7|t3_3qrwlq|t3_3qv1jx|t3_3qv3al|t3_3qwc20|t3_3qx74u|t3_3qx7jo|t3_3qxxwv|t3_3r0jxa|t3_3r13ry|t3_3r1sz9|t3_3r27x8|t3_3r2uhg|t3_3r2xjv|t3_3r3ngh|t3_3r3z3w|t3_3r4ciw|t3_3r4fav|t3_3r4wkl|t3_3r4xs1|t3_3r525t|t3_3r6isv|t3_3r6n5w|t3_3r73g7|t3_3r7ffq|t3_3r7fpy|t3_3r7fu0|t3_3r7t9g|t3_3r7u37|t3_3r83my|t3_3r8bhn|t3_3r8ctf|t3_3r8hzo|t3_3r8vfp|t3_3r9ctf|t3_3r9h

In [11]:
search_strings_RS = []
first = 0
links_RS = []
for link in links:
    links_RS.append(link.replace("t3_",""))
for last in last_for_month: 
    search_strings_RS.append(re.compile("|".join(links_RS[first:last])))
    first = last
search_strings_RS[11]

re.compile(r'55awms|55b45b|55bo2i|55bshi|55c2o6|55c3ek|55c42e|55cen2|55cfri|55chh4|55ct9c|55cthm|55cwvv|55d97o|55df95|55dxvx|55dyqr|55e35p|55e4wo|55eeio|55eimr|55f4a7|55f73x|55fcic|55fjw1|55fl1c|55fpmq|55frsv|55fx2r|55g47w|55h07o|55h789|55h7jw|55hc8a|55hhy2|55hp3n|55iatg|55irx9|55iu6m|55j1wg|55j3s1|55j4ak|55j6o1|55j8b7|55ja3f|55jbtb|55jetg|55jgp2|55jn33|55joxb|55jsxz|55jud1|55k1tu|55koqb|55l1np|55l708|55lg1s|55m0ij|55mfdq|55mgzr|55mm72|55mnzm|55mp2l|55mrvn|55mx9u|55n20x|55n2ny|55n32y|55n3y4|55n4o1|55n65p|55nape|55nf7e|55njm5|55nlf0|55nmo8|55noqr|55npof|55nru2|55nt8i|55nulh|55nv5h|55nw4p|55ok9l|55ov3q|55oyvk|55oz17|55p8py|55pnwr|55ppov|55q996|55qich|55qps9|55qu9s|55qxre|55qykh|55qzwg|55r0qj|55r98z|55rjrm|55rkq4|55rl4d|55rmv2|55rsym|55rvrr|55rwn9|55rzpg|55shu9|55sjcw|55sx25|55t06m|55t2n9|55t4r2|55t6bl|55t9j9|55titw|55tjsj|55tmya|55tpu5|55ts0u|55tu3a|55tx9h|55tyz3|55tz65|55ulwj|55unxj|55urkw|55uv05|55uw9q|55v2g2|55v8ob|55vjpk|55vv4o|55w24g|55wao7|55wo6s|55xeq1|55xv3r|55y79u|55y7jm|55y8sf|

### commented out to prevent replacing existing files
# generate files with all comments from the relevant links
i = 0
for file in filesAll[12:24,1]:
    
    new_file = file.replace(".bz2", "-db.json")
    new_file = new_file.replace("data/raw/", "data/")
    
    print(new_file)
    
    with open(new_file,'w') as w:

        #progress = 0
        start = datetime.datetime.now()

        with bz2.open(file, 'rt') as input_:
            for line in input_:
                progress = progress + 1
                link = json.loads(line)["id"]
                result = search_strings_RS[i].search(link)
                if result != None:
                    w.write(line)    
    
    i = i + 1
    print(datetime.datetime.now()-start)

In [20]:
add_t3 = lambda link: "t3_" + link

In [33]:
last_for_month[11]-last_for_month[10]

1109

In [ ]:
# check if we got all links in RC files
for i in range(0,12):
    filename = "data/RS_"+periods[i]+"-db.json"
    print(filename)
    df = read_json_as_pandas(filename)
    df["link_id"] = df["id"].apply(add_t3)
    dflist = df["link_id"].tolist()
    dfset = set(dflist)
    dfsize = len(dfset)
    print(dfsize)
    if i == 0:
        print(last_for_month[i]-71)
    else:
        print(last_for_month[i]-last_for_month[i-1])

In [67]:
# check if we got all links in RS files
for i in range(0,12):
    filename = "data/RS_"+periods[i]+"-db.json"
    print(filename)
    df = read_json_as_pandas(filename)
    df["link_id"] = df["id"].apply(add_t3)
    dflist = df["link_id"].tolist()
    dfset = set(dflist)
    dfsize = len(dfset)
    print(dfsize)
    if i == 0:
        print(last_for_month[i]-71)
    else:
        print(last_for_month[i]-last_for_month[i-1])

data/RS_2015-11-db.json
745
746
data/RS_2015-12-db.json
561
562
data/RS_2016-01-db.json
695
695
data/RS_2016-02-db.json
787
787
data/RS_2016-03-db.json
821
821
data/RS_2016-04-db.json
797
797
data/RS_2016-05-db.json
834
834
data/RS_2016-06-db.json
868
868
data/RS_2016-07-db.json
969
969
data/RS_2016-08-db.json
1040
1040
data/RS_2016-09-db.json
1051
1051
data/RS_2016-10-db.json
1109
1109
